In [24]:
pip install dash pandas plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
pip install jupyter-dash dash plotly pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import sqlite3
from jupyter_dash import JupyterDash

# 📌 Conectar a SQLite y cargar los datos
sqlite_db_path = "accidentes.sqlite"
conn = sqlite3.connect(sqlite_db_path)
df = pd.read_sql_query("SELECT anio, accident_severity, condiciones_climaticas FROM accidentes", conn)
conn.close()

# 📌 Inicializar la aplicación Dash en Jupyter Notebook
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])

# 📌 Opciones de filtros
years = sorted(df["anio"].dropna().unique())
severity_options = df["accident_severity"].unique()

# 📌 Diseño del Dashboard en Jupyter
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("🚗 Dashboard de Accidentes", className="text-center text-light mb-4"),

            # 📌 Barra lateral con filtros
            dbc.Card([
                html.H4("🔍 Filtros", className="text-center"),
                html.Label("Selecciona el Año:"),
                dcc.Dropdown(id="year-filter", options=[{"label": y, "value": y} for y in years], multi=True),
                html.Br(),
                html.Label("Selecciona la Severidad:"),
                dcc.Dropdown(id="severity-filter", options=[{"label": s, "value": s} for s in severity_options], multi=True),
            ], body=True, className="mb-4")
        ], width=3),

        # 📌 Área de gráficos
        dbc.Col([
            dbc.Row([
                dbc.Col(dbc.Card([dcc.Graph(id="accidents-severity")], body=True), width=6),
                dbc.Col(dbc.Card([dcc.Graph(id="accidents-weather")], body=True), width=6),
            ]),
            dbc.Row([
                dbc.Col(dbc.Card([dcc.Graph(id="accidents-yearly")], body=True), width=12),
            ]),
        ], width=9)
    ]),
], fluid=True)

# 📌 Callbacks para actualizar gráficos dinámicamente
@app.callback(
    [Output("accidents-severity", "figure"),
     Output("accidents-weather", "figure"),
     Output("accidents-yearly", "figure")],
    [Input("year-filter", "value"),
     Input("severity-filter", "value")]
)
def update_charts(selected_years, selected_severity):
    filtered_df = df.copy()

    if selected_years:
        filtered_df = filtered_df[filtered_df["anio"].isin(selected_years)]
    if selected_severity:
        filtered_df = filtered_df[filtered_df["accident_severity"].isin(selected_severity)]

    fig_severity = px.histogram(filtered_df, x="accident_severity", title="Distribución por Severidad", color="accident_severity")
    fig_weather = px.histogram(filtered_df, x="condiciones_climaticas", title="Accidentes por Clima", color="condiciones_climaticas")
    fig_yearly = px.histogram(filtered_df, x="anio", title="Evolución de Accidentes por Año", color="anio")

    return fig_severity, fig_weather, fig_yearly

# 📌 Ejecutar el Dashboard dentro de Jupyter Notebook
app.run_server(mode="inline")

ModuleNotFoundError: No module named 'dash_bootstrap_components'

In [29]:
import pyodbc
import pandas as pd
from pymongo import MongoClient
import redis
import json

#  Conectar a SQL Server
conn_sql = pyodbc.connect("DRIVER={SQL Server};"
                          "SERVER=127.0.0.1\\SQLEXPRESS;"
                          "DATABASE=AccidentesDB;"
                          "Trusted_Connection=yes;")
cursor_sql = conn_sql.cursor()

#  Conectar a MongoDB
client_mongo = MongoClient("mongodb://localhost:27017/")
db_mongo = client_mongo["AccidentesDB"]
collection_mongo = db_mongo["Accidentes"]

#  Conectar a Redis
redis_client = redis.StrictRedis(host="localhost", port=6379, db=0, decode_responses=True)

In [30]:
query = "SELECT Pais, Año, Mes, Zona, Vehiculos_Involucrados FROM Accidentes"
df_sql = pd.read_sql(query, conn_sql)
print(df_sql.head())  # Verifica los datos

C:\Users\jordy.navarro\AppData\Local\Temp\ipykernel_5220\4119228399.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



     Pais   Año        Mes    Zona  Vehiculos_Involucrados
0     USA  2002    Octubre   Rural                       1
1      UK  2014  Diciembre  Urbano                       3
2     USA  2012      Julio  Urbano                       4
3      UK  2017       Mayo  Urbano                       2
4  Canada  2002      Julio   Rural                       1


In [36]:
#  Intentar obtener datos desde Redis
cached_data = redis_client.get("accidentes_por_pais")

if cached_data:
    df_mongo = pd.DataFrame(json.loads(cached_data))  # Convertir JSON a DataFrame
    print(" Datos recuperados desde Redis")
else:
    #  Si Redis no tiene los datos, consultamos MongoDB
    df_mongo = pd.DataFrame(list(collection_mongo.find({}, {"_id": 0, "Pais": 1, "Vehiculos_Involucrados": 1})))
    
    #  Guardamos los datos en Redis para futuras consultas
    redis_client.set("accidentes_por_pais", json.dumps(df_mongo.to_dict(orient="records")))
    print(" Datos consultados en MongoDB y guardados en Redis")

 Datos recuperados desde Redis


In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px

#  Inicializar la aplicación Dash con Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

#  Gráfico 1: Accidentes por país
fig1 = px.bar(df_sql.groupby("Pais").size().reset_index(name="Total_Accidentes"),
              x="Pais", y="Total_Accidentes", title="Accidentes por País")

#  Gráfico 2: Accidentes por año
fig2 = px.line(df_sql.groupby("Año").size().reset_index(name="Total_Accidentes"),
               x="Año", y="Total_Accidentes", title="Evolución de Accidentes por Año")

# 📌 Gráfico 3: Vehículos involucrados por zona
fig3 = px.pie(df_sql.groupby("Zona")["Vehiculos_Involucrados"].sum().reset_index(),
              names="Zona", values="Vehiculos_Involucrados", title="Distribución de Vehículos Involucrados por Zona")

# 📌 Layout del Dashboard
app.layout = dbc.Container([
    html.H1("Dashboard de Accidentes de Tráfico", className="text-center"),
    
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig1), width=6),
        dbc.Col(dcc.Graph(figure=fig2), width=6),
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig3), width=12),
    ])
], fluid=True)

#  Ejecutar la aplicación
if __name__ == "_main_":
    app.run_server(debug=True)

In [38]:
#  Crear DataFrame con insights
report = pd.DataFrame({
    "Métrica": ["País con más accidentes", "Año con más accidentes", "Zona con más vehículos involucrados"],
    "Valor": [
        df_sql["Pais"].value_counts().idxmax(),
        df_sql["Año"].value_counts().idxmax(),
        df_sql.groupby("Zona")["Vehiculos_Involucrados"].sum().idxmax()
    ]
})

#  Guardar el reporte en CSV
report.to_csv("reporte_accidentes.csv", index=False)

print(" Reporte generado: reporte_accidentes.csv")

 Reporte generado: reporte_accidentes.csv


In [43]:
import json

#  Guardar el reporte en Redis
redis_client.set("reporte_accidentes", json.dumps(report.to_dict(orient="records")))
print(" Reporte almacenado en Redis.")

#  Recuperar el reporte desde Redis
cached_report = redis_client.get("reporte_accidentes")
if cached_report:
    print(" Reporte recuperado desde Redis:", json.loads(cached_report))

 Reporte almacenado en Redis.
 Reporte recuperado desde Redis: [{'Métrica': 'País con más accidentes', 'Valor': 'Canada'}, {'Métrica': 'Año con más accidentes', 'Valor': 2002}, {'Métrica': 'Zona con más vehículos involucrados', 'Valor': 'Rural'}]


In [44]:
import pyodbc
import pandas as pd
import plotly.express as px

#  Conectar a SQL Server
conn = pyodbc.connect("DRIVER={SQL Server};"
                      "SERVER=127.0.0.1\\SQLEXPRESS;"  # Ajusta si tu servidor es diferente
                      "DATABASE=AccidentesDB;"
                      "Trusted_Connection=yes;")

#  Cargar datos desde SQL Server
query = "SELECT Pais, Año, Mes, Zona, Vehiculos_Involucrados FROM Accidentes"
df_sql = pd.read_sql(query, conn)

#  Gráfico 1: Accidentes por país
fig1 = px.bar(df_sql.groupby("Pais").size().reset_index(name="Total_Accidentes"),
              x="Pais", y="Total_Accidentes", title="Accidentes por País")

#  Gráfico 2: Accidentes por año
fig2 = px.line(df_sql.groupby("Año").size().reset_index(name="Total_Accidentes"),
               x="Año", y="Total_Accidentes", title="Evolución de Accidentes por Año")
#  Gráfico 3: Vehículos involucrados por zona
fig3 = px.pie(df_sql.groupby("Zona")["Vehiculos_Involucrados"].sum().reset_index(),
              names="Zona", values="Vehiculos_Involucrados", title="Distribución de Vehículos Involucrados por Zona")

# 📌 Mostrar los gráficos
fig1.show()
fig2.show()
fig3.show()

C:\Users\jordy.navarro\AppData\Local\Temp\ipykernel_5220\2890459299.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

